<a href="https://colab.research.google.com/github/ramyahramzy/Colab/blob/main/VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "numpy==1.26.4" "chromadb" "hnswlib==0.8.0"

  Using cached chromadb-1.0.17-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.3 kB)
  Using cached hnswlib-0.8.0.tar.gz (36 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import numpy, chromadb

print("NumPy:", numpy.__version__)
print("Chroma:", chromadb.__version__)




NumPy: 1.26.4
Chroma: 1.0.17


In [3]:
import datasets

In [4]:
from datasets import load_dataset
ds = load_dataset("sadeem-ai/arabic-qna")
print(ds)

client = chromadb.PersistentClient(path="chroma-ar")
collection = client.create_collection(name="ar_docs", metadata={"hnsw:space":"cosine"})
print("Chroma collection:", collection.name)

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1030 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 1030
    })
})
Chroma collection: ar_docs


In [5]:
!pip install datasets faiss-cpu==1.7.4 sentence-transformers==2.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 907.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
from datasets import load_dataset

qna_dataset = load_dataset("sadeem-ai/arabic-qna")

In [8]:
qna_dataset = qna_dataset.filter(lambda example: example["has_answer"]== True)

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1030 [00:00<?, ? examples/s]

In [ ]:
qna_dataset

In [10]:
doc_text = qna_dataset["train"]["text"][:]

In [11]:
doc_text[10]

'حاصل على الدكتوراه في تخصص النمذجة الرياضية للأنظمة الكيميائية الدقيقة من جامعة ليهاي الولايات المتحدة الأمريكية تخصص الهندسة الكيميائية عام 2005.'

In [12]:
meta_data = [

    {
        "source":rec["source"],
        "title":rec["title"]
    }
    for rec in qna_dataset["train"]
]

In [15]:
meta_data[50]

{'source': 'https://ar.wikipedia.org/wiki?curid=8379033',
 'title': 'جدول ميداليات الألعاب الأولمبية الصيفية 2020'}

In [13]:
doc_ids = [
    str(i)
    for i in range(len(doc_text))
]

In [14]:
from sentence_transformers import SentenceTransformer

In [15]:
model_id = "sentence-transformers/distiluse-base-multilingual-cased-v2"
device = "cuda:0"

model = SentenceTransformer(model_id).to(device)


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [28]:
doc_text

Column(['اعتمادًا على المكان الذي ينتمي إليه الفرد، قد يكون تصورهم للوقت هو أن «الساعة تحكم اليوم» أو «سنصل إلى هناك عندما نصل إلى هناك». تحسين احتمالات النجاح في المجتمع العالمي يتطلب فهم الاختلافات الثقافية والتقاليد وأنماط الاتصال.', 'وبعد عدة أسابيع، تأخذ اليرقة مكانـًا لها (على نبتة أو على الأرض، حسب نوع الفراشة) ومن ثم تتحول اليرقة إلى خادرة. وفي هذه المرحلة تبدو الخادرة وكأنها جسم صامت – علبة أو درنة (وهذا أيضًا متعلق بنوع الفراشة). فهي لا تأكل ولا تتحرك من مكانها (لكنها كائن حي). وفي داخل العلبة تحدث تطورات معينة وفي نهاية الأمر تخرج الفراشة من الخادرة.', 'كان توزيع المرض بين الأطفال الذين تتراوح أعمارهم بين 5-9 سنوات من أعلى معدل في قاعدة البيانات التي تم تحليلها، تليها النساء في سن 60-64. كانت نسبة الإصابة بين المرضى الإناث أعلى من تلك بين المرضى الذكور في جميع الفئات العمرية. ووفقًا للبيانات المتعلقة بالاتجاهات الاجتماعية، فمن المرجح أن تمتلك النساء قطًا فوق الرجال ؛ مما يدعم ارتفاع معدلات الإصابة بهذا المرض لدى النساء. يزيد خطر الإصابة مع زيادة عدد القطط المقيمة في المنزل. 

In [16]:
encoded_doc = model.encode(doc_text,show_progress_bar=True)

Batches:   0%|          | 0/127 [00:00<?, ?it/s]

In [38]:
doc_text[10] , encoded_doc[10]

('حاصل على الدكتوراه في تخصص النمذجة الرياضية للأنظمة الكيميائية الدقيقة من جامعة ليهاي الولايات المتحدة الأمريكية تخصص الهندسة الكيميائية عام 2005.',
 array([-0.00827503,  0.02703172, -0.0498628 , -0.05056059, -0.00715023,
         0.01376726, -0.01454557, -0.02480279, -0.00047609, -0.01045045,
         0.03023715, -0.02332602, -0.01728397, -0.03127356,  0.01887566,
        -0.02433516, -0.00328179,  0.02666473,  0.03013171, -0.00949634,
        -0.05833136,  0.04672301, -0.05709622, -0.06757797,  0.01133902,
        -0.00813907,  0.01069323,  0.02491183, -0.05644623,  0.00968405,
         0.02276987, -0.00397209,  0.00232533,  0.049471  ,  0.0376363 ,
         0.0147678 , -0.00333426,  0.02307388,  0.02232517, -0.02466129,
        -0.04907871,  0.01817251, -0.03494669, -0.01811384,  0.05829418,
        -0.10454393,  0.02031245, -0.01067277,  0.02680674,  0.04748166,
         0.04696179,  0.02541605,  0.00758474, -0.03070022, -0.02502944,
        -0.00381934, -0.00362914, -0.00486981,

## ChromaDB

In [17]:
chroma_client = chromadb.PersistentClient(path="./chromadb-ar-docs")

In [18]:
collection = chroma_client.create_collection(name="ar_docs", metadata={"hnsw:space":"cosine"})

In [19]:
collection.add(
    documents=doc_text,
    metadatas=meta_data,
    ids=doc_ids,
    embeddings=encoded_doc
)